**Questions and things I am working on:**

**Questions:**

-No big questions right now. I am making good progress and am very happy with how things have been going. I implemented template generation and created a fake signal (I know the signal I generated is unrealistic; I wanted one that was easily visible on a graph). I also added some code to find an SNR so that when I add quantum stuff, I can compare the results. I have most of the quantum code in my notebook so far, as it helps me understand it better if I write it there. 

**Tasks (and why they are not done yet lol):**

-Transfer data to quantum 

-Building Quantum Circuits ~ It's tough, but I am beginning to understand it



In [36]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister # Visualises the quantum circuits, creates the quantum register, and creates the classical register
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator, Aer # Allows for more advanced simulations to be run - For GPU usage, CUDA must be installed and that is not open source :(
from qiskit import transpile # From what I understand, this helps rewrite the program so it can run on hardware more easily
import matplotlib.pyplot as plt # Imports matplotlib so we can visulase the templates, not nessaray, but I want to see them :)
import numpy as np # For number operations
import h5py # For using HDF5 stroage
import os # To play with the file system
import glob as glob # Helps with file functions with calssical matched filtering

Function the genorates the templates

In [67]:
# Funtion to create the template wave
def generate_cw_template(f0, fdot, alpha, delta, t_ref, duration, dt, h0=1e-24, f_plus=1.0, f_cross=0.0, phi0=0.0):
    # Creates an array of times from 0 to the spesified duration spaced by dt, this is the smapled data times
    times = np.arange(0, duration, dt)
    # Centers the time axis around the reference time
    t = times - t_ref

    # Computes the signal phase at each time
    phase = 2 * np.pi * (f0 * t + 0.5 * fdot * t**2) + phi0
    #Constructsvespertine the full signal h(t) 
    signal = h0 * (f_plus * np.cos(phase) + f_cross * np.sin(phase))
    # Stores all he parameters used in generating the wave, important for reproducibility
    metadata = {
        #I did impliment the other stuff, it is just not here
        'f0': f0, # Initial fewquency of the GW (Hz)
        'fdot': fdot, # First frequency derivate also known as the spin down(Hz/s)
        'alpha': alpha, # Sky position of source, used in metadata
        'delta': delta, # Sky position of source, used in metadata
        't_ref': t_ref, # Reference time (start of observation)_
        'h0': h0, # Amplitude of the gravitainal wave
        'phi0': phi0, #initial offset of the signal
        'duration': duration, # Total length of the template in seconds
        'dt': dt #time step (the interval for sampling)
    }
    #Returns the time array, generated waveform array, all the metadata
    return times, signal, metadata

Saves the created template to a file and has a main funcutin

In [68]:
if __name__ == "__main__":
    #This is just the shortend version, more discription is shown in Create_Data
    f0_values = np.linspace(0, 5, 6) # Looks at a range of frequencys - Format (min #, max #, # of steps)
    fdot_values = np.linspace(-1e-10, -5e-10, 5)  # The derivitve of above, frequency per second - Format (min #, max #, # of steps)
    alpha = 1.5      
    delta = 0.3      
    t_ref = 0        
    duration = 86400  
    dt = 1.0         

    
    output_dir = "cw_templates" #assigns the folder the templaes should go into
    
    os.makedirs(output_dir, exist_ok=True) # does stuff with the os
    
    # Runs thoguht all of the values defined above and creates templates with them
for f0 in f0_values:
    for fdot in fdot_values:
        times, signal, metadata = generate_cw_template(f0, fdot, alpha, delta, t_ref, duration, dt)
        filename = os.path.join(output_dir, f"template_f{f0:.3f}_fdot{fdot:.1e}.h5")
        save_template_hdf5(filename, times, signal, metadata)
        print(f"Saved: {filename}")
    
    print(f"Template saved to {filename}") # Prints the filename


Saved: cw_templates/template_f0.000_fdot-1.0e-10.h5
Saved: cw_templates/template_f0.000_fdot-2.0e-10.h5
Saved: cw_templates/template_f0.000_fdot-3.0e-10.h5
Saved: cw_templates/template_f0.000_fdot-4.0e-10.h5
Saved: cw_templates/template_f0.000_fdot-5.0e-10.h5
Template saved to cw_templates/template_f0.000_fdot-5.0e-10.h5
Saved: cw_templates/template_f1.000_fdot-1.0e-10.h5
Saved: cw_templates/template_f1.000_fdot-2.0e-10.h5
Saved: cw_templates/template_f1.000_fdot-3.0e-10.h5
Saved: cw_templates/template_f1.000_fdot-4.0e-10.h5
Saved: cw_templates/template_f1.000_fdot-5.0e-10.h5
Template saved to cw_templates/template_f1.000_fdot-5.0e-10.h5
Saved: cw_templates/template_f2.000_fdot-1.0e-10.h5
Saved: cw_templates/template_f2.000_fdot-2.0e-10.h5
Saved: cw_templates/template_f2.000_fdot-3.0e-10.h5
Saved: cw_templates/template_f2.000_fdot-4.0e-10.h5
Saved: cw_templates/template_f2.000_fdot-5.0e-10.h5
Template saved to cw_templates/template_f2.000_fdot-5.0e-10.h5
Saved: cw_templates/template_f3

Matched Filerting - Classical Code

In [5]:
def load_hdf5_signal(filename):
    with h5py.File(filename, 'r') as f:
        times = f['times'][:]
        signal = f['signal'][:]
        metadata = dict(f['metadata'].attrs)
    return times, signal, metadata


def matched_filter(template, data, noise_std):
    """
    Simple matched filter assuming stationary white Gaussian noise.
    Computes normalized dot product (matched filter SNR).
    """
    norm = np.linalg.norm(template)
    if norm == 0:
        raise ValueError("Template has zero norm.")
    template_norm = template / norm

    snr = np.correlate(data, template_norm, mode='valid') / noise_std
    return snr


Calls the Classical Matched Filterign code

In [6]:
# Load data and template
data_times, noisy_data, data_meta = load_hdf5_signal("CW/wave_with_noise.h5")
_, template, _ = load_hdf5_signal("CW/wave_no_noise.h5")

# Estimate noise level from Saved: cw_templates/template_f5.000_fdot-2.0e-10.h5Saved: cw_templates/template_f5.000_fdot-2.0e-10.h5m data (or use known std)
estimated_noise_std = np.std(noisy_data - template)  # or known value


template_files = glob.glob("cw_templates/template_f*.h5")
max_snr = 0
best_template = None

for tf in template_files:
    _, template, _ = load_hdf5_signal(tf)
    snr = matched_filter(template, noisy_data, estimated_noise_std)
    peak = np.max(np.abs(snr))
    if peak > max_snr:
        max_snr = peak
        best_template = tf

print(f"Best template: {best_template}")
print(f"Peak SNR: {max_snr}")

Best template: cw_templates/template_f3.000_fdot-1.0e-10.h5
Peak SNR: 227.82104370068575


Design the Quantum Ciructs 

In [35]:
#Create the quantum circuits that will do the matched filtering - This will likely entail using QFT (Quantum Fourier Transforms) or another highly researched technique for quantum computing 


#Divide and conquer algorithm may help put the data in Quantum form
#There are many pre-made circuits at https://quantum.cloud.ibm.com/docs/en/api/qiskit/circuit_library that may be helpful, including QFT

Encode the data using Matched Filtering - Quantum

In [3]:
#Mutply the templates by the acctual data - This is relitivly simple once the circut has been designed

Transfer the results from the quantum to the calssical register

In [ ]:
#Use Qiskit to transfer data from the quantum to the classical register so we can perform data processing on it and other operations.
#I think I understand this piece of code, I just need more pieces put in place before I can implement it

Construct and add the products together - Classical

In [ ]:
#See if any of the results have a good SNR

Do the mesurement calculations - Classical